In [41]:
import polars as pl

# --- CONFIG ---
DATA_FILE_1 = "./results/data/pressure_vessel_baseline_gen150_runs500_cthigh_psize6.parquet"
DATA_FILE_2 = "./results/data/pressure_vessel_relaxed_gen150_runs500_cthigh_psize6.parquet"
# DATA_FILE_2 = DATA_FILE_1
RUN_A = 1
RUN_B = 2

SIGNATURE_COLS = ["f_1_min"]
GEN_COL = "generation"
RUN_COL = "run"
ROUND_DECIMALS = 6

df1 = pl.read_parquet(DATA_FILE_1)
df2 = pl.read_parquet(DATA_FILE_2)

# round floats to stabilize equality checks
df1 = df1.with_columns(
    [pl.col(c).round(ROUND_DECIMALS).alias(c) for c in SIGNATURE_COLS if df1.schema[c] in (pl.Float32, pl.Float64)]
)
df2 = df2.with_columns(
    [pl.col(c).round(ROUND_DECIMALS).alias(c) for c in SIGNATURE_COLS if df2.schema[c] in (pl.Float32, pl.Float64)]
)

dfa = df1.filter(pl.col(RUN_COL) == RUN_A)
dfb = df2.filter(pl.col(RUN_COL) == RUN_B)

rows = []

for g in dfa[GEN_COL].unique().sort():
    A = (
        dfa.filter(pl.col(GEN_COL) == g)
        .select(SIGNATURE_COLS)
        .unique()
        .with_columns(pl.struct(SIGNATURE_COLS).hash().alias("h"))
        .select("h")
        .sort("h")
    )

    B = (
        dfb.filter(pl.col(GEN_COL) == g)
        .select(SIGNATURE_COLS)
        .unique()
        .with_columns(pl.struct(SIGNATURE_COLS).hash().alias("h"))
        .select("h")
        .sort("h")
    )

    identical = (A.height == B.height) and (A["h"].to_list() == B["h"].to_list())

    rows.append(
        {
            "generation": g,
            "identical": identical,
            "n_unique_A": A.height,
            "n_unique_B": B.height,
        }
    )

result = pl.DataFrame(rows)

display(result)

print(f"\nIdentical generations: {result.filter(pl.col('identical')).height} / {result.height}")

print(dfa.filter(pl.col(GEN_COL) == 1))
print(dfb.filter(pl.col(GEN_COL) == 1))

generation,identical,n_unique_A,n_unique_B
i64,bool,i64,i64
1,false,6,6
2,false,6,6
3,false,6,6
4,false,6,6
5,false,6,6
…,…,…,…
146,false,6,3
147,false,6,6
148,false,5,6



Identical generations: 0 / 150
shape: (12, 13)
┌───────────┬───────────┬────────────┬────────────┬───┬──────────────┬────────────┬─────┬──────────┐
│ x_1       ┆ x_2       ┆ x_3        ┆ x_4        ┆ … ┆ f_1_min      ┆ generation ┆ run ┆ seed     │
│ ---       ┆ ---       ┆ ---        ┆ ---        ┆   ┆ ---          ┆ ---        ┆ --- ┆ ---      │
│ f64       ┆ f64       ┆ f64        ┆ f64        ┆   ┆ f64          ┆ i32        ┆ i32 ┆ i32      │
╞═══════════╪═══════════╪════════════╪════════════╪═══╪══════════════╪════════════╪═════╪══════════╡
│ 77.274629 ┆ 38.071392 ┆ 36.93346   ┆ 67.956551  ┆ … ┆ 35426.734165 ┆ 1          ┆ 1   ┆ 28012027 │
│ 35.345201 ┆ 82.34062  ┆ 42.863146  ┆ 187.899337 ┆ … ┆ 34938.720743 ┆ 1          ┆ 1   ┆ 28012027 │
│ 3.867281  ┆ 7.90759   ┆ 96.647115  ┆ 19.614608  ┆ … ┆ 8609.226246  ┆ 1          ┆ 1   ┆ 28012027 │
│ 61.793662 ┆ 61.825626 ┆ 174.819051 ┆ 135.249325 ┆ … ┆ 324938.70961 ┆ 1          ┆ 1   ┆ 28012027 │
│           ┆           ┆            ┆     